In [ ]:
import numpy as np
import pandas as pd
import emoji


from keras.models import Sequential
from keras.layers import Dense, LSTM, SimpleRNN, Embedding
from keras_preprocessing.text import Tokenizer
from keras_preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from sklearn.model_selection import train_test_split  # Import for splitting the data
from gensim.models import KeyedVectors
from datetime import datetime

In [ ]:
data = pd.read_csv("data/emoji_data/emoji_data.csv", header=None)
data.head()

Emoji Dictionary

In [ ]:
"""emoji_dictionary = {
    0: ":red_heart:",  # Love
    1: ":face_with_tears_of_joy:",  # Laughter
    2: ":smiling_face_with_heart-eyes:",  # Adoration
    3: ":loudly_crying_face:",  # Sadness
    4: ":fire:",  # Excitement
    5: ":thumbs_up:",  # Approval
    6: ":folded_hands:",  # Gratitude
    7: ":angry_face:",  # Anger
    8: ":sparkles:",  # Happiness
    9: ":weary_face:",  # Exhaustion
    10: ":astonished_face:",  # Surprise
    11: ":confused_face:",  # Confusion
    12: ":tropical_drink:",  # Celebration
    13: ":broken_heart:",  # Heartbreak
    14: ":thinking_face:",  # Contemplation
    15: ":sleeping_face:",  # Sleepiness
    16: ":victory_hand:",  # Success
    17: ":thumbs_down:",  # Disapproval
    18: ":grimacing_face:",  # Discomfort
    19: ":smiling_face_with_halo:",  # Innocence
}"""

emoji_dictionary = {
    0: ":red_heart:",  # Love #
    1: ":face_with_tears_of_joy:",  # Laughter
    2: ":grinning_face_with_big_eyes:", # Happiness #
    3: ":loudly_crying_face:",  # Sadness #
    4: ":smiling_face_with_heart-eyes:",  # Adoration
    5: ":fire:",  # Excitement
    6: ":thumbs_up:",  # Approval
    7: ":folded_hands:",  # Gratitude
    8: ":angry_face:",  # Anger
    9: ":thinking_face:",  # Contemplation
}


Function to get labels from CLDR names

In [ ]:
def label_to_emoji(label):
    return emoji.emojize(emoji_dictionary[label])

Reading data

In [ ]:
data = pd.read_csv('data/emoji_data/emoji_data.csv', header=None)
data.head()

In [ ]:

def label_to_emoji(label):
    return emoji.emojize(emoji_dictionary[label])

In [ ]:
X = data[0].values
Y = data[1].values


Embeddings

With glove dataset

In [ ]:
# with open('data/glove_dataset/glove.6B.100d.txt','r', encoding='utf8') as file:
#     content = file.readlines()

In [ ]:
fasttext_model_path = "data/fast_text/crawl_dataset/crawl-300d-2M-subword.vec"
fasttext_model = KeyedVectors.load_word2vec_format(fasttext_model_path)


In [263]:
# Convert FastText embeddings to a dictionary
embeddings = {}
for word in fasttext_model.index_to_key:
    embeddings[word] = fasttext_model.get_vector(word)


In [264]:
# embeddings = {}

# for line in content:
#     line = line.split()
#     embeddings[line[0]] = np.array(line[1:], dtype=float)

With crawl dataset

In [ ]:
fasttext_model_path = "data/fast_text/crawl_dataset/crawl-300d-2M-subword.vec"
fasttext_model = KeyedVectors.load_word2vec_format(fasttext_model_path)


In [ ]:
# Convert FastText embeddings to a dictionary
embeddings = {}
for word in fasttext_model.index_to_key:
    embeddings[word] = fasttext_model.get_vector(word)


Convert input text into tokens

In [ ]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(X)
word_to_index = tokenizer.word_index


In [ ]:
Xtokens = tokenizer.texts_to_sequences(X)


In [ ]:
def get_maxlen(data):
    maxlen = 0
    for sent in data:
        maxlen = max(maxlen, len(sent))
    
    return maxlen
maxlen = get_maxlen(Xtokens)


In [ ]:
Xtrain = pad_sequences(Xtokens, maxlen=maxlen, padding='post', truncating='post')


In [ ]:
Ytrain = to_categorical(Y)

Split data into train and test set

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(Xtrain, Ytrain, test_size=0.2, random_state=42)

Model

In [ ]:
embed_size = 300

In [ ]:
embedding_matrix = np.zeros((len(word_to_index) + 1, embed_size))

for word, i in word_to_index.items():
    if word in embeddings:
        embed_vector = embeddings[word]
        embedding_matrix[i] = embed_vector
    else:
        # Handle out-of-vocabulary words or phrases by aggregating subword embeddings
        phrase_embed_sum = None
        for subword in word.split():
            if subword in embeddings:
                if phrase_embed_sum is None:
                    phrase_embed_sum = embeddings[subword]
                else:
                    phrase_embed_sum += embeddings[subword]
        if phrase_embed_sum is not None:
            # Take the average of subword embeddings
            embedding_matrix[i] = phrase_embed_sum / len(word.split())

In general more layers >>

In [ ]:

epochs = 1000


#units
layers = [50, 100, 23, 67, 30, 20, 5, 10, 8, 9, 2, 256, 5]

num_layers = len(layers)


num_classes = len(emoji_dictionary)

Code to generate file name for saving model

In [ ]:
def returnfilename():
    layers_str = '_'.join(map(str, layers))  # Convert layers to string and join them with '_'
    return f"epochs_{epochs}_layers_{layers_str}"


In [ ]:
model = Sequential()

# Add Embedding layer
model.add(Embedding(input_dim=len(word_to_index) + 1,
                    output_dim=embed_size,
                    input_length=maxlen,
                    weights=[embedding_matrix],
                    trainable=False))


# Add LSTM layers
for i in range(num_layers):
    # For the last layer, return_sequences=False
    if i == num_layers - 1:
        model.add(LSTM(units=layers[i]))
    else:
        model.add(LSTM(units=layers[i], return_sequences=True))

# Add output Dense layer
model.add(Dense(num_classes, activation='softmax'))


model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
model.fit(Xtrain, Ytrain, epochs=epochs,  verbose = 0)

In [ ]:
filename = returnfilename()

model_json = model.to_json()
with open(f"models/json/{filename}.json", "w") as json_file:
    json_file.write(model_json)
    
# serialize weights to HDF5
model.save_weights(f"models/weights/{filename}.h5")

In [ ]:
metrics_folder = "models/metrics"

In [ ]:
# Evaluate the model on the test set
loss, accuracy = model.evaluate(X_test, Y_test)
print("Test Loss:", loss)
print("Test Accuracy:", accuracy)
current_datetime = datetime.now()
formatted_datetime = current_datetime.strftime('%d %B, %Y')

# Write model architecture and details to file
with open(f"{metrics_folder}/{filename}.md", "w", encoding="utf-8") as file:
    file.write("# Model description:<br>\n")
    file.write(f"Date generated: {formatted_datetime}<br>\n")
    file.write(f"Number of classes: {num_classes}<br>\n")
    file.write(f"Number of samples: {len(X)}<br>\n")
    file.write(f"Training set size: {len(X_train)}<br>\n")
    file.write(f"Test set size: {len(X_test)}<br>\n")
    file.write(f"Epochs used: {epochs}<br>\n")
    file.write(f"Number of layers: {num_layers}<br>\n")
    file.write(f"Layers used: {layers}<br>\n")
    file.write("Optimizer: Adam<br>\n")
    file.write("Loss function: Categorical Crossentropy<br>\n")
    file.write("# Evaluation Results<br>\n")
    file.write(f"Test Loss: {loss:.4f}<br>\n")
    file.write(f"Test Accuracy: {accuracy:.4f}<br><br>\n")

# Make predictions on the test set
y_pred = model.predict(X_test)


In [ ]:
# Make predictions on the test set
y_pred = model.predict(X_test)

# Convert predicted and actual labels from one-hot encoded format to integer labels
y_pred_labels = np.argmax(y_pred, axis=1)
Y_test_labels = np.argmax(Y_test, axis=1)

# Compare predicted labels with actual labels
correct_predictions = np.sum(y_pred_labels == Y_test_labels)
total_predictions = len(Y_test_labels)
accuracy = correct_predictions / total_predictions

# Print accuracy
print("Accuracy:", accuracy)
# Write accuracy to file
with open(f"{metrics_folder}/{filename}.md", "a", encoding="utf-8") as file:
    file.write("Accuracy: {:.4f}\n\n".format(accuracy))


# Write misclassified samples to file
with open(f"{metrics_folder}/{filename}.md", "a", encoding="utf-8") as file:
    file.write("## Misclassified samples:<br>\n")
    for i in range(len(y_pred_labels)):
        if y_pred_labels[i] != Y_test_labels[i]:
            file.write(
                f"Predicted: {label_to_emoji(y_pred_labels[i])} Actual: {label_to_emoji(Y_test_labels[i])}<br>\n"
            )



for i in range(len(y_pred_labels)):
    if y_pred_labels[i]!=Y_test_labels[i]:
        print("Predicted:", label_to_emoji(y_pred_labels[i]), "Actual:", label_to_emoji(Y_test_labels[i]))


In [ ]:
# Make predictions
test = ["I am trying", "I want to cry", "This is just sad"]
test_seq = tokenizer.texts_to_sequences(test)
Xtest = pad_sequences(test_seq, maxlen=maxlen, padding='post', truncating='post')
y_pred = model.predict(Xtest)
y_pred = np.argmax(y_pred, axis=1)

# Write predictions to file
with open(f"{metrics_folder}/{filename}.md", "a", encoding="utf-8") as file:
    file.write("\n## Random test predictions: (accuracy based on discussion)<br>\n")
    for i in range(len(test)):
        file.write(f"{test[i]} {label_to_emoji(y_pred[i])}<br>\n")

In [ ]:


for i in range(len(test)):
    print(test[i], label_to_emoji(y_pred[i]))